In [4]:
# import pandas
# from pandas_datareader import data as pdr
# import yfinance as yfin
# yfin.pdr_override()
import backtrader
import pandas as pd
import pandas_datareader.data as web
import datetime
import backtrader as bt
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10, 6) # (w, h)

from datetime import datetime
import backtrader as bt

class SmaCross(bt.SignalStrategy):
    def __init__(self):
        sma1, sma2 = bt.ind.SMA(period=10), bt.ind.SMA(period=30)
        crossover = bt.ind.CrossOver(sma1, sma2)
        self.signal_add(bt.SIGNAL_LONG, crossover)

cerebro = bt.Cerebro()
cerebro.addstrategy(SmaCross)

0

In [2]:
# from pandas_datareader.yahoo.daily import YahooDailyReader as ydr
# import requests

# USER_AGENT = {
#     'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
#                    ' Chrome/91.0.4472.124 Safari/537.36')
#     }
# sesh = requests.Session()
# sesh.headers.update(USER_AGENT)

# yd = ydr(symbols="VFINX",start='2021-01-01', session=sesh)
# df = yd.read()
# yd.close()

# yd = ydr(symbols="VUSTX",start='2021-01-01', session=sesh)
# df2 = yd.read()
# yd.close()

In [1]:
def backtest(datas, strategy, plot=False, **kwargs):
    global cerebro
    cerebro = bt.Cerebro(stdstats=False)
    for data in datas:
        cerebro.adddata(data)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
    cerebro.addanalyzer(bt.analyzers.Returns)
    cerebro.addanalyzer(bt.analyzers.DrawDown)
    cerebro.addstrategy(strategy, **kwargs)
    results = cerebro.run()
    if plot:
        cerebro.plot()
    return (results[0].analyzers.drawdown.get_analysis()['max']['drawdown'],
            results[0].analyzers.returns.get_analysis()['rnorm100'],
            results[0].analyzers.sharperatio.get_analysis()['sharperatio'])


In [ ]:
class AssetAllocation(bt.Strategy):
    params = (
        ('equity',0.6),
    )
    
    def __init__(self):
        self.UPRO = self.datas[0]
        self.TQQQ = self.datas[1]
        self.SQQQ = self.datas[2]
        self.prev_TQQQ = []
        self.counter = 0
        
    def next(self):
        is_dip = len(self.prev_TQQQ) > 0 and (self.TQQQ.close / np.mean(self.prev_TQQQ) <= 0.97)
        if  self.counter % 10 == 0 and (not is_dip): #rebalance every 10 workdays
            self.order_target_percent(self.UPRO, target=self.params.equity)
            self.order_target_percent(self.TQQQ, target=(1 - self.params.equity))
        if is_dip:
            self.order_target_percent(self.UPRO, target=self.params.equity + 0.2)
            self.order_target_percent(self.SQQQ, target=(1 - self.params.equity - 0.2))
            print("is_dip")
        self.prev_TQQQ += [self.TQQQ.close]
        self.counter += 1


In [6]:
upro = bt.feeds.YahooFinanceData(dataname='UPRO', fromdate=datetime(2017, 1, 1),todate=datetime(2021, 10, 1))
tqqq = bt.feeds.YahooFinanceData(dataname='TQQQ', fromdate=datetime(2017, 1, 1),todate=datetime(2021, 10, 1))
sqqq = bt.feeds.YahooFinanceData(dataname='SQQQ', fromdate=datetime(2017, 1, 1),todate=datetime(2021, 10, 1))

In [ ]:
dd, cagr, sharpe = backtest([upro, tqqq, sqqq], AssetAllocation, plot=True, equity=0.6)
print(f"Max Drawdown: {dd:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")

In [100]:
df = pd.DataFrame()
class to_CSV(bt.Strategy):
    params = (
        ('equity',0.6),
    )
    
    def __init__(self):
        global df
        self.df = pd.DataFrame()
        self.UPRO = self.datas[0]
#         self.TQQQ = self.datas[1]
#         self.SQQQ = self.datas[2]
        
    def next(self):
        pass
cerebro = bt.Cerebro(stdstats=False)
for data in [upro]: cerebro.adddata(data)
# cerebro.addstrategy(to_CSV)

cerebro.addwriter(bt.WriterFile, out='abc.csv', csv=True)
results = cerebro.run()

In [101]:
import os
datapath = os.path.join(os.getcwd(), 'abc.csv')

In [117]:
a = pd.read_csv(datapath).iloc[1]

In [118]:
a.name

('1',
 'UPRO',
 '1',
 '2017-01-03 23:59:59.999989',
 '27.41',
 '27.68',
 '26.99',
 '27.46',
 '7298336.0',
 '0.0',
 '27.464451',
 'Strategy',
 '1')

In [81]:
data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        fromdate=datetime(2017, 1, 1),todate=datetime(2021, 10, 1),
        reverse=False)

In [92]:
import datetime
import backtrader.feeds as btfeeds
class MyOHLC(btfeeds.GenericCSVData):
    params = (
        ('fromdate', datetime.datetime(2017, 1, 1)),
        ('todate', datetime.datetime(2021, 10, 1)),
        ('nullvalue', 0.0),
        ('dtformat', ('%Y-%m-%d')),
        ('tmformat', ('%H.%M.%S')),
        ('datetime', 1),
        ('open', 2),
        ('high', 3),
        ('low', 4),
        ('close', 5),
        ('volume', 6),
        ('openinterest', -1)
    )
data = MyOHLC(dataname=datapath)
print(data.getfeed())

None


In [93]:
data = bt.feeds.GenericCSVData(
    dataname=datapath,
    fromdate=datetime.datetime(2017, 1, 1),
    todate=datetime.datetime(2021, 10, 1),
    nullvalue=0.0,
    dtformat=('%Y-%m-%d'),
    datetime=0,
    open = 1,
    high = 2,
    low = 3,
    close = 4,
    volume =5, 
    openinterest=-1,
    reverse=False)

In [98]:
df = pd.DataFrame()
class to_CSV(bt.Strategy):
    params = (
        ('equity',0.6),
    )
    
    def __init__(self):
        global df
        self.df = pd.DataFrame()
        self.UPRO = self.datas[0]
        self.TQQQ = self.datas[1]
        self.SQQQ = self.datas[2]
        
    def next(self):
        global a 
        a = self.UPRO
        print(a.close[0])
        return
cerebro = bt.Cerebro(stdstats=False)
for data in [data]: cerebro.adddata(data)
# cerebro.addstrategy(to_CSV)

cerebro.addwriter(bt.WriterFile, out='abc2.csv', csv=True)
results = cerebro.run()

ValueError: time data 'Id' does not match format '%Y-%m-%d'